In [1]:
import numpy as np
import pandas as pd

from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import cross_val_score
from sklearn.feature_selection import SelectKBest
from sklearn.grid_search import GridSearchCV
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cross_validation import train_test_split

In [2]:
dataset = pd.read_csv('dota_games.csv', index_col=0)

In [3]:
dataset.dtypes.value_counts()

float64    221
object       2
int64        1
dtype: int64

### Duplicate Radient/Dire games

In [4]:
def radient_win_pc(extract):
    counts = extract['winner'].value_counts()
    return (1.0 * counts['Radiant Victory']) / (counts['Radiant Victory'] + counts['Dire Victory'])

In [5]:
dataset['winner'].value_counts()

Radiant Victory    3788
Dire Victory       2959
dtype: int64

In [6]:
radient_win_pc(dataset)

0.56143471172372905

In [7]:
data_temp = dataset.copy()
def remap_col_name(col):
    if col.startswith('radient-'):
        col = col.replace('radient-', 'dire-')
    elif col.startswith('dire-'):
        col = col.replace('dire-', 'radient-')
    return col
        
data_temp.columns = [remap_col_name(col) for col in data_temp.columns]
data_temp.loc[dataset['winner'] == 'Radiant Victory', 'winner'] = 'Dire Victory'
data_temp.loc[dataset['winner'] == 'Dire Victory', 'winner'] = 'Radiant Victory' 
# dataset = pd.concat([dataset, data_temp])

In [8]:
dataset = dataset.take(np.random.permutation(len(dataset)))

In [9]:
X = dataset.drop(['winner', 'duration', 'match_id', 'game_mode'], axis=1)
y = dataset['winner']

## MultinominalNB

In [10]:
pipe = Pipeline([    
    ('classifier', MultinomialNB())
])

scores = cross_val_score(pipe, X, y, cv = 10)
scores.mean()

0.6798627088161967

## MultinominalNB + tf-idf

In [11]:
pipe = Pipeline([
    ('tfidf', TfidfTransformer()),
    ('classifier', MultinomialNB())
])

scores = cross_val_score(pipe, X, y, cv = 10)
scores.mean()

0.66237352568992203

## Logistic Regression

In [12]:
pipe = Pipeline([
    ('classifier', LogisticRegression(penalty='l1'))
])

scores = cross_val_score(pipe, X, y, cv = 10)
scores.mean()

0.68505295211481687

## Logistic Regression + tf-idf

In [13]:
pipe = Pipeline([
    ('tfidf', TfidfTransformer()),
    ('classifier', LogisticRegression(penalty='l1'))
])

scores = cross_val_score(pipe, X, y, cv = 10)
scores.mean()

0.68564642653773655

In [14]:
est = LogisticRegression()
est.fit(X, y)

X.columns[np.ravel(np.abs(est.coef_) > 1)]

Index([u'dire-alchemist', u'dire-oracle', u'dire-undying'], dtype='object')

In [15]:
radient_win_pc(dataset[dataset['radient-alchemist'] == 1])

0.36622807017543857

# Neural Net 

In [18]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD

ImportError: No module named keras.models

In [ ]:
from k

In [ ]:


model = Sequential()
model.add(Dense(64, 2, init='uniform'))
model.add(Activation('softmax'))

model.compile(loss='mse', optimizer='sgd')

'''
Demonstration of verbose modes 1 and 2
'''
model.fit(X_train, y_train, nb_epoch=3, batch_size=16, verbose=1)

## Neurolab

In [16]:
# import neurolab as nl

# y_binary = (y == 'Radiant Victory').reshape(-1, 1)
# x_train, x_test, y_train, y_test = train_test_split(X, y_binary, test_size=0.33)

# inp = x_train
# tar = y_train

# net = nl.net.newff([[0,1] for i in range(220)], [50, 1])
# net.trainf = nl.train.train_rprop
# errors = net.train(inp, tar, show=10, epochs=100, lr=0.1)

In [17]:
# # 586.94683040374218 error = 0.74 accuracy

# from sklearn.metrics import accuracy_score
# y_pred = net.sim(x_test)
# y_pred[y_pred < 0.5] = 0
# y_pred[y_pred >= 0.5] = 1

# accuracy_score(y_test, y_pred)